In [60]:
# Simple model, which train embedding for each ingredient and than perform simple softmax classification.
import json, re
from collections import UserDict, Counter
from tqdm import tqdm
from keras.models import Model
from keras.layers import Input, Dense, Embedding, MaxPooling1D, Reshape, AveragePooling2D, Flatten, Dropout
import random, keras
import numpy as np
from keras.callbacks import EarlyStopping

MAX_INGREDIENT_DIM = 8
MAX_INGREDIENT_COUNT = 16

In [74]:
class BiDict(UserDict):
    def __setitem__(self, key, value):
        self.data[key] = value
        self.data[value] = key
    
INGREDIENTS, CUISINES = BiDict(), BiDict()
WORDS = BiDict()
INGREDIENTS_IDS = []

def normalize_name(name):
    name = name.lower()
    name = name.replace('’', "'")
    name = re.sub("[®™!]", ' ', name)
    name = re.sub('\(.+\)', ' ', name)
    if ',' in name:
        name, *rest = name.split(',')
    name = re.sub('\s+', ' ', name)
    
    if not re.fullmatch('[- \'a-zèçîíúéâ&0-9%.]+', name):
        print(name)
        return None
    return name.strip()

records = []
lengths = Counter()
words = Counter()

with open('../input/train.json') as file:
    for record in json.load(file):
        cuisine_id = CUISINES.setdefault(record['cuisine'], len(CUISINES)//2)
        ingredients = []
        for ingredient in record['ingredients']:
            ingredient = normalize_name(ingredient)
            if ingredient is None:
                continue
            
            if ingredient not in INGREDIENTS:
                ingredient_words = ingredient.split(' ')#[:MAX_INGREDIENT_DIM]
                ingredient_ids = [WORDS.setdefault(i, len(WORDS)//2) for i in ingredient_words]
                INGREDIENTS[ingredient] = len(INGREDIENTS_IDS)
                INGREDIENTS_IDS.append(ingredient_ids)
            
            ingredients.append(INGREDIENTS[ingredient])
        lengths[len(ingredients)] += 1
        records.append((cuisine_id, ingredients))
        
print(len(records))
print(len(WORDS)//2)
print(lengths.most_common())

hellmannâ€ or best food canola cholesterol free mayonnais
2 1/2 to 3 lb. chicken
2 1/2 to 3 lb. chicken
39774
3083
[(9, 3753), (10, 3677), (8, 3555), (11, 3512), (7, 3330), (12, 3146), (13, 2698), (6, 2661), (14, 2254), (5, 1892), (15, 1808), (16, 1439), (17, 1160), (4, 1128), (18, 879), (19, 610), (3, 549), (20, 504), (21, 313), (22, 218), (2, 193), (23, 141), (24, 91), (25, 72), (26, 46), (28, 27), (1, 22), (29, 21), (27, 20), (30, 15), (31, 11), (32, 4), (36, 4), (33, 4), (40, 3), (34, 3), (35, 3), (38, 2), (49, 2), (65, 1), (52, 1), (59, 1), (43, 1)]


In [75]:
ingredients_count = 5
emb_dim = 32
output_classes = 20

a = Input(shape=(MAX_INGREDIENT_COUNT, MAX_INGREDIENT_DIM))

b = Embedding(len(WORDS)//2, emb_dim)(a)
b = Dropout(rate=0.1)(b)
b = AveragePooling2D(pool_size=(1,MAX_INGREDIENT_DIM))(b)
# b = MaxPooling2D(pool_size=(1, MAX_INGREDIENT_DIM))(emb)
b = Reshape((-1,emb_dim))(b)
b = MaxPooling1D(pool_size=(MAX_INGREDIENT_COUNT,))(b)
b = Flatten()(b)
# b = Dense(512, activation='tanh')(b)
# b = Dense(128, activation='tanh')(b)
# b = Dropout(rate=0.1)(b)
b = Dense(128, activation='relu')(b)
b = Dropout(rate=0.1)(b)

d = Dense(output_classes, activation='softmax')(b)

model = Model(inputs=a, outputs=d)
model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 16, 8)             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 16, 8, 32)         98656     
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 8, 32)         0         
_________________________________________________________________
average_pooling2d_5 (Average (None, 16, 1, 32)         0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 16, 32)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 32)             0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 32)                0         
__________

In [76]:
def get_data(n):
    inputs, outputs = [], []
    for i in range(n):
        cuisine, items = random.choice(records)
        items = random.choices(items, k=MAX_INGREDIENT_COUNT)
        items = [random.choices(INGREDIENTS_IDS[i], k=MAX_INGREDIENT_DIM) for i in items]
        outputs.append(cuisine)
        assert all(len(x) == MAX_INGREDIENT_DIM for x in items)
        inputs.append(items)
        
    return np.array(inputs), keras.utils.to_categorical(np.array(outputs), num_classes=output_classes)

# print(get_data(5))
bsize = 2**16

batch0, batch1 = None, get_data(bsize)
for i in range(2):
    batch0 = batch1
    batch1 = get_data(bsize)
    
    model.fit(*batch0, verbose=1, epochs=50, batch_size=2**10, validation_data=batch1, callbacks=[EarlyStopping(patience=2)])

Train on 65536 samples, validate on 65536 samples
Epoch 1/50
65536/65536 [==============================] - 6s 88us/step - loss: 2.7658 - acc: 0.1806 - val_loss: 2.5447 - val_acc: 0.3231
Epoch 2/50
65536/65536 [==============================] - 6s 85us/step - loss: 2.3509 - acc: 0.3211 - val_loss: 2.0688 - val_acc: 0.4153
Epoch 3/50
65536/65536 [==============================] - 5s 83us/step - loss: 1.7659 - acc: 0.5076 - val_loss: 1.5367 - val_acc: 0.5571
Epoch 4/50
65536/65536 [==============================] - 5s 78us/step - loss: 1.4022 - acc: 0.5914 - val_loss: 1.3027 - val_acc: 0.6175
Epoch 5/50
65536/65536 [==============================] - 5s 75us/step - loss: 1.2207 - acc: 0.6380 - val_loss: 1.1770 - val_acc: 0.6522
Epoch 6/50
65536/65536 [==============================] - 5s 81us/step - loss: 1.1127 - acc: 0.6666 - val_loss: 1.0998 - val_acc: 0.6720
Epoch 7/50
65536/65536 [==============================] - 5s 78us/step - loss: 1.0381 - acc: 0.6865 - val_loss: 1.0449 - val_acc

In [78]:
weights = model.layers[1].get_weights()[0]
print(weights.shape)
w2=np.sqrt(np.sum(weights*weights, axis=1))
print(w2.shape)

for i in range(0,20):
    w = (np.matmul(weights, weights[i]) / w2)/w2[i]
    w[i] = 0
    b = np.argmax(w)
    
    print(WORDS[i], WORDS[b], w[b])

# print(WORDS[''], np.sum(weights[0]))

(3083, 32)
(3083,)
romaine e-fu 0.58718914
lettuce garlic 0.7340528
black zucchini 0.73226404
olives vine 0.64733803
grape anise 0.59557295
tomatoes bell 0.7219744
garlic salt 0.7721785
pepper bay 0.7187047
purple salt 0.76434726
onion minced 0.68547773
seasoning ben's 0.4888513
garbanzo buckwheat 0.59822273
beans black 0.6009315
feta crumbles 0.8137418
cheese pizza 0.65302044
crumbles feta 0.81374186
plain crumbles 0.5743107
flour lard 0.6877942
ground lentils 0.7262362
salt juice 0.78417885


In [100]:
VOTES = 16
total = correct = 0

with open('../input/test.json') as file, open('output.csv', 'w') as outputf:
    for record in tqdm(json.load(file)):
        ingredients_ids = []
        for ingredient in record['ingredients']:
            ingredient = normalize_name(ingredient)
            if ingredient is None:
                continue
                
            ingredient = ingredient.split(' ')
            
            ingredient = [WORDS[i] for i in ingredient if i in WORDS]
            
            if len(ingredient):
                ingredients_ids.append(ingredient)
                
        qs = []
        for i in range(VOTES):
            igs = random.choices(ingredients_ids, k=MAX_INGREDIENT_COUNT)
            igs = [random.choices(j, k=MAX_INGREDIENT_DIM) for j in igs]
            qs.append(igs)
#         print(qs)
        y = model.predict(np.array(qs))
#         print(y.shape)
        y = np.sum(y, axis=0)
#         print(y.shape)
#         print(y)
        cuisine = CUISINES[np.argmax(y)]
#         if cuisine == record['cuisine']:
#             correct += 1
#         total += 1
#         break
        outputf.write(f"{record['id']},{cuisine}\n")
# print(100.0*correct/total)


 73%|███████▎  | 7272/9944 [00:15<00:05, 526.21it/s]


100%|██████████| 9944/9944 [00:22<00:00, 451.25it/s]